# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from rentcast_api import *

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

%load_ext autoreload
%autoreload 2

In [ ]:
rcp = RentCastPlotter.open_db(db_path='./Data/All')

In [ ]:
rcp.list_all_cities()

In [ ]:
list(rcp.data_processed['beaverton_or'])

In [ ]:
city_states = [['Portland', 'OR'],
               ['Happy Valley', 'OR'],
               ['Beaverton', 'OR'],
               ['Camas', 'WA'],
               ['Lake Oswego', 'OR'],
               ['Tigard', 'OR'],
               ['Gresham', 'OR'],
               ['Oregon City', 'OR'],
               ['Hood River', 'OR'],]

filters = [
    ("squareFootage", ">", 1000),
    ("squareFootage", "<", 4000),
    ("price_per_sqft", ">", 100),
    ("price_per_sqft", "<", 1000),
]

rcp.plot_city_states(city_states, filters=filters)

# ML

In [ ]:
rcp.data_all.head()

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.cluster import KMeans
# from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
# Set up data
df = rcp.data_all

# Example feature set
num_cols = ["bedrooms", "bathrooms", "months"]
cat_cols = ["city", "propertyType", "geo_cluster"]

num_cols = ["bedrooms", "bathrooms", "months"]
cat_cols = ["city", "geo_cluster", 'sale_month']

# Target
target = "lastSalePrice"

# Drop rows missing target
df = df.dropna(subset=[target] + num_cols)

# Filter properties that are on the edges
df_clean = df[
    (df["lastSalePrice"] < 2_000_000) &
    (df["bedrooms"].between(1, 8)) &
    (df["bathrooms"].between(1, 8)) &
    (df["squareFootage"].between(500, 6000)) &
    (df["lotSize"].between(500, 25_000)) &
    (df["yearBuilt"].between(1850, 2025)) &
    (df["lastSalePrice"] / df["squareFootage"] < 2000)
]

df_clean.reset_index(drop=True, inplace=True)
df_clean = copy.deepcopy(df_clean)

# Ensure it's string and split
df_clean[["sale_month" ,"sale_year"]] = df_clean["month-year"].str.split("-", expand=True)
df_clean['sale_year'] = df_clean['sale_year'].values.astype(int)

# Convert to integers
df_clean["sale_month"] = df_clean["sale_month"].astype(int)
# df_clean["sale_year"] = df_clean["sale_year"].astype(int) - 1985

# Get rough neighborhood
coords = df_clean[["latitude","longitude"]]
df_clean["geo_cluster"] = KMeans(n_clusters=50, random_state=42).fit_predict(coords)
cluster_means = df_clean.groupby("geo_cluster")["lastSalePrice"].median()


# Normalize some of the data
# df_clean['house_age'] = 2025 - df_clean['yearBuilt']
# calculated_cols = ['log_lotSize', 'log_sqft', 'log_sqFt_per_bedroom', 'log_sqFt_per_bathroom', 'yearsOld', 'sale_month']
calculated_cols = ['log_lotSize', 'log_sqft', 'yearsOld', 'log_cluster_time_median', 'lotsqft_per_sqft']
num_cols += calculated_cols
df_clean['log_lotSize'] = np.log(df_clean['lotSize'].values)
df_clean['log_sqft'] = np.log(df_clean['squareFootage'].values)
df_clean['lotsqft_per_sqft'] = np.log(df_clean['lotSize'].values / df_clean['squareFootage'].values)
df_clean['log_sqFt_per_bedroom'] = np.log(df_clean['squareFootage'].values) / df_clean['bedrooms'].values
df_clean['log_sqFt_per_bathroom'] = np.log(df_clean['squareFootage'].values) / df_clean['bathrooms'].values
df_clean['yearsOld'] = 2025 - df_clean['yearBuilt'].values
# df_clean.drop(columns=['yearBuilt', 'lotSize'], inplace=True)

y = np.log1p(df_clean[target])  

for cluster_id, group in df_clean.groupby("geo_cluster"):
    X_temp = group[["months"]].values
    y_temp = group[target].values

    # simple linear fit
    model = LinearRegression()
    model.fit(X_temp, y_temp)

    # predict for this group's rows
    df_clean.loc[group.index, "log_cluster_time_median"] = model.predict(X_temp)

X = df_clean[num_cols + cat_cols]

# Better as a target, error is in % rather than abs
# 100k error is different for a 200k home vs a 20M home


print(list(X))

In [ ]:
# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ]
)

# Model
# xgb = XGBRegressor(
#     n_estimators=2000,      # number of boosting rounds (like max_iter)
#     learning_rate=0.03,     # step size shrinkage
#     max_depth=12,           # depth of each tree
#     subsample=0.8,          # use 80% of rows per tree (helps generalization)
#     colsample_bytree=0.8,   # use 80% of features per tree
#     random_state=42,
#     n_jobs=-1               # use all cores
# )

hgb = HistGradientBoostingRegressor(
    max_iter=400, 
    learning_rate=0.01, 
    max_depth=10,
    early_stopping=True,
    random_state=42
)
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", hgb)
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit
model.fit(X_train, y_train)

# Cross-validation score
scores = cross_val_score(model, X_train, y_train, cv=5,
                         scoring="neg_root_mean_squared_error")
print("CV RMSE:", -np.mean(scores))

# Evaluate on test
test_score = model.score(X_test, y_test)  # R^2
print("Test R²:", test_score)

# Prediction

In [ ]:
import numpy as np

feature_cols = num_cols + cat_cols + ['sale_year']

# pick a row by index (from the same schema as training)
i = np.random.randint(0, df_clean.shape[0])
x_one = df_clean.loc[i:i+0, feature_cols]          # note the double brackets to keep it 2D
pred_log = model.predict(x_one)[0]         # model outputs log-price
pred_price = float(np.expm1(pred_log))     # back to dollars
act_price = df_clean.loc[i, 'lastSalePrice']
per_error = (pred_price - act_price) / act_price * 100

print("Predicted - Actual - %% Error --- $%i - $%i - %.1f%%" % (pred_price / 1000, act_price / 1000, per_error))
df_clean.loc[i:i+0, feature_cols + ['lastSalePrice']]

In [ ]:
import numpy as np

feature_cols = num_cols + cat_cols

x_one = df_clean.loc[:, feature_cols]               # note the double brackets to keep it 2D
actual_prices = df_clean.loc[:, 'lastSalePrice'] 
pred_log = model.predict(x_one)                     # model outputs log-price
pred_prices = np.expm1(pred_log)                    # back to dollars

In [ ]:
plt.plot(np.array(pred_prices) / 1e3, np.array(actual_prices) / 1e3, lw=0, ms=4, mec='black', alpha=0.02, marker='o')
plt.grid(True)

ax = plt.gca()
# get limits
lims = [
    min(ax.get_xlim()[0], ax.get_ylim()[0]),
    max(ax.get_xlim()[1], ax.get_ylim()[1]),
]

# plot 1:1 line
ax.plot(lims, lims, 'k--', alpha=1)  # black dashed line
ax.set_xlim(lims)
ax.set_ylim(lims)

plt.ylabel('Actual Cost [$1k]')
plt.xlabel('Predicted Cost [$1k]')


# Plot Residuals

In [ ]:
per = False

feature_cols = num_cols + cat_cols

x_one = df_clean.loc[:, feature_cols]               # note the double brackets to keep it 2D
actual_prices = df_clean.loc[:, 'lastSalePrice'].values 
pred_log = model.predict(x_one)                     # model outputs log-price
pred_prices = np.expm1(pred_log)   

if per:
    error = (pred_prices - actual_prices) / actual_prices * 100
else:
    error = (pred_prices - actual_prices) / 1000

important_cols = ['months', 'squareFootage', 'sale_year', 'yearsOld', 'lastSalePrice', 'geo_cluster']

plt.figure(figsize=(12, 6))

for idx, important_col in enumerate(important_cols):

    col_data = df_clean.loc[:, important_col].values

    if important_col == 'lastSalePrice':
        col_data = col_data / 1e3

    plt.subplot(2, 3, idx+1)
    plt.plot(col_data, error, lw=0, ms=4, marker='o', mec='black', alpha=0.005)
    plt.xlabel(important_col)
    plt.grid()
    if per:
        plt.title('Error [%%] vs %s' % important_col)
        plt.ylim([-200, 200])
    else:
        plt.title('Error [$1k] vs %s' % important_col)

plt.tight_layout()

In [ ]:
import numpy as np
import pandas as pd

def binned_residual_stats(x, err, bins=20, strategy="quantile"):
    """
    x: 1D array-like feature
    err: 1D array-like residuals (e.g., (pred - actual)/actual*100 or pred-actual)
    bins: number of bins
    strategy: "quantile" (equal-count bins) or "uniform" (equal-width bins)
    Returns: DataFrame with bin centers, mean, p10, p90, count
    """
    x = pd.Series(x).astype(float)
    err = pd.Series(err).astype(float)

    if strategy == "quantile":
        # quantile bins handle skewed features better
        q = np.linspace(0, 1, bins + 1)
        edges = x.quantile(q).values
        # ensure strictly increasing (dedup if constant segments)
        edges = np.unique(edges)
        if len(edges) < 3:  # too few unique edges
            edges = np.linspace(x.min(), x.max(), min(bins, 10) + 1)
        cats = pd.cut(x, bins=edges, include_lowest=True)
    else:
        cats = pd.cut(x, bins=bins)

    g = pd.DataFrame({"x": x, "err": err, "bin": cats}).dropna().groupby("bin")
    stats = g.agg(
        x_mid=("x", lambda s: (s.min() + s.max()) / 2.0),
        err_mean=("err", "mean"),
        err_p10=("err", lambda s: np.percentile(s, 10)),
        err_p90=("err", lambda s: np.percentile(s, 90)),
        count=("err", "size"),
    ).reset_index(drop=True)

    return stats.sort_values("x_mid")


import matplotlib.pyplot as plt

def plot_binned_residuals(x, err, feature_name, bins=20, strategy="quantile",
                          ylim=None, ax=None):
    """
    Plots mean residual per bin with a shaded 10–90% band.
    """
    stats = binned_residual_stats(x, err, bins=bins, strategy=strategy)
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))

    ax.axhline(0, ls="--", lw=1, color="gray")
    ax.fill_between(stats["x_mid"], stats["err_p10"], stats["err_p90"],
                    alpha=0.2, edgecolor="none")
    ax.plot(stats["x_mid"], stats["err_mean"], marker="o", lw=2, mec='black')
    ax.set_xlabel(feature_name)
    ax.set_ylabel("Mean error")
    if ylim is not None:
        ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    return ax, stats

features_to_check = [
    ("months", df_clean["months"].values),
    ("squareFootage", df_clean["squareFootage"].values),
    ("lotSize", df_clean["lotSize"].values),
    ("yearsOld", df_clean["yearsOld"].values),
    ("lastSalePrice", actual_prices),
    ("geoCluster", df_clean["geo_cluster"].values),
]

fig, axes = plt.subplots(2, 3, figsize=(12, 7))
axes = axes.flatten()

for ax, (name, feat) in zip(axes, features_to_check):
    plot_binned_residuals(feat, error, name, bins=25, strategy="quantile",
                          ylim=(-300, 300), ax=ax)

# Hide any extra subplot
for ax in axes[len(features_to_check):]:
    ax.axis("off")

fig.suptitle("Binned residuals (mean ± 10–90% band)", y=1.02)
plt.tight_layout()

# Checks

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

feature_cols = num_cols + calculated_cols
PartialDependenceDisplay.from_estimator(model, X_test.iloc[:1_000, :], feature_cols)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 6))

feature_cols = num_cols + cat_cols + calculated_cols

for idx, col in enumerate(feature_cols):

    plt.subplot(2, 4, idx+1)
    df_clean[col].hist(bins=30, edgecolor='black')
    print("Median %s: %.2f, Max %s: %.2f" % (col, df_clean[col].median(), col,  df_clean[col].max()))

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_test_mean = np.full_like(y_test, y_train.mean())
baseline_rmse = mean_squared_error(y_test, y_test_mean)
print("Baseline RMSE:", baseline_rmse)

In [ ]:
print(df_clean[["squareFootage","lotSize","yearBuilt"]].describe())
print(df_clean["squareFootage"].value_counts().head())

In [ ]:
from sklearn.linear_model import LinearRegression
toy = LinearRegression()
toy.fit(df_clean[["squareFootage"]], df_clean["lastSalePrice"])
print("R² (sqft only):", toy.score(df_clean[["squareFootage"]], df_clean["lastSalePrice"]))

In [ ]:
print(y_train.mean(), y_test.mean())
print(X_train["city"].value_counts().head())
print(X_test["city"].value_counts().head())